In [20]:
import torch

#Define device as MPS if available
device = torch.device('mps' if torch.has_mps else 'cpu')
print(device)

mps


/var/folders/vs/4nfsm6sj3zqgsrt1l65q891w0000gn/T/ipykernel_17191/2497146413.py:4: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device('mps' if torch.has_mps else 'cpu')


In [21]:
torch.backends.mps.is_built()

True

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small').to(device)


In [23]:
import pandas as pd

#Loading dataset
data = pd.read_csv('/Users/saitejagudidevini/Documents/costcobills/ ./Sample_YouTube_Comments_without_Sentiment_Labels.csv')
comments = data['comment'].tolist()
replies = data['reply'].tolist()

#Checking the data
data.head(50)

,comment,reply
0,"Thanks for the tutorial, it helped a lot!",I'm glad you found it informative!
1,Can you create more tutorials like this?,Good question! It compares well in certain sce...
2,Could you explain step 4 in more detail?,Glad this met your needs!
3,Can you suggest additional resources?,"I'll keep creating more tutorials, thank you!"
4,I didn't understand the last part.,Glad this met your needs!
5,Which libraries did you use?,Check out these resources for more information...
6,Can you create more tutorials like this?,Good question! It compares well in certain sce...
7,Can you create more tutorials like this?,Check out these resources for more information...
8,I didn't understand the last part.,I chose this method because...
9,Can you suggest additional resources?,I chose this method because...


In [24]:
from torch.utils.data import Dataset, DataLoader

# Custom dataset class
class CommentsDataset(Dataset):
    def __init__(self, comments, replies, tokenizer, max_length=128):
        self.comments = comments
        self.replies = replies
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        # Tokenize input (comment) and target (reply)
        inputs = self.tokenizer(
            self.comments[idx],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        labels = self.tokenizer(
            self.replies[idx],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Correct names: `input_ids` and `attention_mask`
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        labels = labels['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

# Sample data for testing
comments = ["This tutorial was helpful!", "Can you explain step 3?"]
replies = ["Glad it helped!", "Sure, let me explain step 3 in detail."]

# Instantiate dataset and dataloader
dataset = CommentsDataset(comments, replies, tokenizer)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)


In [25]:
def prepare_data(text):
    #Tokenize input and send tensors to MPS
    inputs = tokenizer(text, return_tensors='pt').to(device)
    return inputs

In [34]:
from torch.optim import AdamW

num_epochs=30
learning_rate = 2e-5

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
tokenizer.pad_token = tokenizer.eos_token

# Training loop for one epoch (example)
for epoch in range(num_epochs):
    for batch in train_loader:  # Assuming you have a DataLoader `train_loader`
        # Move batch data to MPS device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    


    print(f"Epoch {epoch + 1}/{num_epochs} Loss: {loss.item()}")



Epoch 1/3 Loss: 0.07403086870908737
Epoch 2/3 Loss: 0.4264058470726013
Epoch 3/3 Loss: 0.26615872979164124


In [36]:
def generate_reply(comment):
    inputs = tokenizer(comment + tokenizer.eos_token, return_tensors="pt", padding=True)

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    # Generate reply with MPS-enabled model
    reply_ids = model.generate(
        inputs=input_ids, 
        attention_mask=attention_mask, 
        max_length=100, 
        pad_token_id=tokenizer.eos_token_id, 
        temperature=0.7, 
        top_k=50, 
        top_p=0.9, 
        do_sample=True
    )

    reply = tokenizer.decode(reply_ids[0], skip_special_tokens=True)
    return reply


In [37]:
#save the fine_tuned Model

model.save_pretrained("./fine_tuned_dialoGPT")
tokenizer.save_pretrained("./fine_tuned_dialoGPT")

('./fine_tuned_dialoGPT/tokenizer_config.json',
 './fine_tuned_dialoGPT/special_tokens_map.json',
 './fine_tuned_dialoGPT/vocab.json',
 './fine_tuned_dialoGPT/merges.txt',
 './fine_tuned_dialoGPT/added_tokens.json',
 './fine_tuned_dialoGPT/tokenizer.json')

In [38]:
# Test comments
test_comments = [
    "Your work is shit",
    "Always Thankful for your work",
    "Nice",
]

# Generate replies for each test comment
for comment in test_comments:
    reply = generate_reply(comment)
    print(f"Comment: {comment}")
    print(f"Reply: {reply}\n")


Comment: Your work is shit
Reply: Your work is shitI'm in.

Comment: Always Thankful for your work
Reply: Always Thankful for your workJust got it. Thanks!

Comment: Nice
Reply: NiceI have it in black

